In [110]:
#imports
import sys
from bs4 import BeautifulSoup
import requests
import pandas
import time

#import selenium to scrap JS generated pages
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait



In [111]:
def getWeatherForCity(name, lat, long, driver, cities):
    newCity = {}
    newCity['name'] = name
    newCity['lat'] = lat
    newCity['long']= long

    urlPage = "https://cn.bing.com/search?q=" + name + "+china+historical+weather&ensearch=1&FORM=BESBTP"

    #using selenium as a firefox browser


    driver.get(urlPage)
    driver.implicitly_wait(100)



    #scrap the weather data with BS


    content = BeautifulSoup(driver.page_source, "html.parser")
    title = content.find("span", {"class":"b_prominentFocusLabel"})

    timer = 0
    while (title == None and timer < 30):
        time.sleep(1)
        content = BeautifulSoup(driver.page_source, "html.parser")
        title = content.find("span", {"class":"b_prominentFocusLabel"})
        timer = timer + 1



    if(title is not None and name +',' in title.text):

        historical = driver.execute_script("return HistoricalWeather.data.mAvgs;")

        while(historical == {}):
            time.sleep(1)
            historical = driver.execute_script("return HistoricalWeather.data.mAvgs;")

        for i in range(0,12):
            newCity.update(historical[i].copy());
            cities[i].append(newCity.copy())
    else:
        #No weather data
        print("No data for "+ name)
        for i in range(0,12):
            cities[i].append(newCity.copy())

In [112]:
#setting up selenium driver

driver = webdriver.Firefox()

#initialize citiesList
cities = []
for i in range(0,12):
    cities.append([])



In [113]:
#get the list of chinese cities from xls file
citiesList = pandas.read_excel("citiesLoc.xlsx", sheetname=0, header=0, index_col=False, keep_default_na=True).to_dict()


In [114]:
#scrapping the data for each city
nbCities = len(citiesList)
i = 0
for cityName, loc in citiesList.items():
    print(str(i)+"/"+str(nbCities))
    getWeatherForCity(cityName, loc[0], loc[1], driver, cities)
    i=i+1
    


0/828
1/828
2/828
3/828
4/828
5/828
No data for Chadian
6/828
7/828
No data for Luxu
8/828
9/828
No data for Yanjiang
10/828
No data for Dawukou
11/828
No data for Tianfu
12/828
No data for Qiancheng
13/828
14/828
No data for Boli
15/828
No data for Shizilu
16/828
No data for Panshan
17/828
No data for Nanding
18/828
No data for Zhangjiakou Shi Xuanhua Qu
19/828
No data for Xiangyang
20/828
21/828
No data for Shilaoren
22/828
No data for Kashgar
23/828
24/828
25/828
No data for Shahecheng
26/828
27/828
No data for Wacheng
28/828
No data for Zhenzhou
29/828
No data for Luocheng
30/828
31/828
32/828
No data for Dongling
33/828
34/828
No data for Huai'an
35/828
36/828
No data for Weining
37/828
38/828
39/828
40/828
No data for Dengying
41/828
No data for Kuai’an
42/828
No data for Dongyuan
43/828
44/828
45/828
46/828
No data for Chamdo
47/828
48/828
No data for Dazuo
49/828
50/828
51/828
No data for Qiaojiang
52/828
53/828
No data for Qingnian
54/828
55/828
56/828
57/828
No data for Suile

In [115]:
#saving in xls file
for i in range(0,12):
    df = pandas.DataFrame(cities[i])
    df.to_excel("RetrievedData/citiesAvrWeather"+cities[i][0]['mShort']+".xlsx",index=False, encoding="utf-8")